<a href="https://colab.research.google.com/github/anirbansen3027/SecureAndPrivateAI/blob/master/10_encrypted_deep_learning_using_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install syft

In [0]:
#importing the libraries
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim
import torch.nn.functional as F

W0803 19:05:04.323811 140396472612736 hook.py:98] Torch was already hooked... skipping hooking process


In [0]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

W0803 19:05:04.337252 140396472612736 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0803 19:05:04.341140 140396472612736 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior
W0803 19:05:04.344824 140396472612736 base.py:628] Worker me already exists. Replacing old worker which could cause                     unexpected behavior


In [0]:
# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# A Toy Model
model = Net()

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        
train()

tensor(2.0279)
tensor(2.2203)
tensor(2.8373)
tensor(1.1780)
tensor(0.7678)
tensor(0.3863)
tensor(0.2809)
tensor(0.2036)
tensor(0.1494)
tensor(0.1088)
tensor(0.0792)
tensor(0.0580)
tensor(0.0443)
tensor(0.0367)
tensor(0.0358)
tensor(0.0388)
tensor(0.0518)
tensor(0.0668)
tensor(0.1067)
tensor(0.1431)


In [0]:
#predictions
model(data)

tensor([[-0.1538],
        [-0.1785],
        [ 0.7567],
        [ 0.6568]], grad_fn=<AddmmBackward>)

In [0]:
#creating a encrypted model and sharing it among alice, bob and secure worker
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [0]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:35329391769 -> alice:2748789077]
 	-> (Wrapper)>[PointerTensor | me:18466816438 -> bob:11515801655]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:62188038877 -> alice:31423194293]
 	-> (Wrapper)>[PointerTensor | me:90761529266 -> bob:42032718081]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:18797672796 -> alice:90263370972]
 	-> (Wrapper)>[PointerTensor | me:85486716593 -> bob:49652079112]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:13070099284 -> alice:26624377606]
 	-> (Wrapper)>[PointerTensor | me:78284133230 -> bob:290350875

In [0]:
#creating an encrypted dataset and sharing it among alice, bob and secure_Worker
encrypted_data = data.fix_precision().share(alice,bob,crypto_provider=secure_worker)

In [0]:
#getting the encrypted prediction
encrypted_prediction = encrypted_model(encrypted_data)

In [0]:
#getting the prediction in decrypted format
prediction = encrypted_prediction.get().float_precision()

In [0]:
prediction

tensor([[-0.1530],
        [-0.1770],
        [ 0.7560],
        [ 0.6570]])